<img style="float: left;;" src='https://github.com/gdesirena/Procesamiento_Natural_del_Lenguaje_2024/blob/main/Modulo%20II/Figures/alinco.png?raw=1' /></a>

# Modulo I: Regresión Logística para Análisis de Sentimientos

## Importar librerías y funciones

In [ ]:
import nltk
from os import getcwd

In [70]:
# agregue la carpeta, tmp2, desde nuestro espacio de trabajo local que contiene archivos de corpus descargados previamente a la ruta de datos de nltk
# esto permite importar estos archivos sin descargarlos nuevamente cuando actualizamos nuestro espacio de trabajo
filePath = f"{getcwd()}/../tmp2"
nltk.data.path.append(filePath)

In [72]:
import numpy as np
import pandas as pd
from nltk.corpus import twitter_samples

from utils import process_tweet

### Prepara los datos
* `twitter_samples` contiene subconjuntos de 5,000 tweets positivos, 5,000 tweets negativos y el conjunto completo de 10,000 tweets.
     * Si utiliza los tres conjuntos de datos, introduciríamos duplicados de los tweets positivos y negativos.
     * Seleccionará solo los cinco mil tweets positivos y los cinco mil tweets negativos.

In [73]:
# select the set of positive and negative tweets
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

In [74]:
len(%colors)

5000

* Train test split: 20% para test, y 80% para train.


In [75]:
# split the data into two pieces, one for training and one for testing (validation set)

#Test y train para tweets positivos
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
#Test y train para tweets negativos

test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

# Datos de entrenamiento
train_x = train_pos + train_neg
test_x = test_pos + test_neg


* Creear una matriz de etiquetas positivas y negativas.

In [77]:
# combinar tweets positivos y negativos
train_y = np.append(np.ones((len(train_pos),1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos),1)), np.zeros((len(test_neg),1)), axis=0)

In [80]:
# imprimir test y train
len(test_y), len(train_y)

(2000, 8000)

In [81]:
# crear el diccionario de frecuencias
from utilss import Utilities as prep
a = prep()


This is the Utilities Constructor


In [82]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [83]:
freqs = a.build_freqs(train_x, train_y)

In [84]:
freqs

{('followfriday', 1.0): 23,
 ('top', 1.0): 30,
 ('engag', 1.0): 7,
 ('member', 1.0): 14,
 ('commun', 1.0): 27,
 ('week', 1.0): 72,
 (':)', 1.0): 2847,
 ('hey', 1.0): 60,
 ('jame', 1.0): 7,
 ('odd', 1.0): 2,
 (':/', 1.0): 5,
 ('pleas', 1.0): 80,
 ('call', 1.0): 27,
 ('contact', 1.0): 4,
 ('centr', 1.0): 1,
 ('02392441234', 1.0): 1,
 ('abl', 1.0): 6,
 ('assist', 1.0): 1,
 ('mani', 1.0): 28,
 ('thank', 1.0): 504,
 ('listen', 1.0): 14,
 ('last', 1.0): 39,
 ('night', 1.0): 55,
 ('bleed', 1.0): 2,
 ('amaz', 1.0): 41,
 ('track', 1.0): 5,
 ('scotland', 1.0): 2,
 ('congrat', 1.0): 15,
 ('yeaaah', 1.0): 1,
 ('yipppi', 1.0): 1,
 ('accnt', 1.0): 2,
 ('verifi', 1.0): 2,
 ('rqst', 1.0): 1,
 ('succeed', 1.0): 1,
 ('got', 1.0): 57,
 ('blue', 1.0): 8,
 ('tick', 1.0): 1,
 ('mark', 1.0): 1,
 ('fb', 1.0): 4,
 ('profil', 1.0): 2,
 ('15', 1.0): 4,
 ('day', 1.0): 187,
 ('one', 1.0): 90,
 ('irresist', 1.0): 2,
 ('flipkartfashionfriday', 1.0): 16,
 ('like', 1.0): 187,
 ('keep', 1.0): 55,
 ('love', 1.0): 336,
 

### Procesamiento del tweet

La función dada `process_tweet ()` tokeniza el tweet en palabras individuales, elimina las palabras vacías y aplica la derivación.

In [85]:
process_tweet(train_x[0])

['followfriday', 'top', 'engag', 'member', 'commun', 'week', ':)']

##  Extrayendo las características

* Dada una lista de tweets, extraiga las características y guárdelas en una matriz. Extraerás dos características.
     * La primera característica es la cantidad de palabras positivas en un tweet.
     * La segunda característica es la cantidad de palabras negativas en un tweet.
* Luego entrene su clasificador de regresión logística en estas características.
* Pruebe el clasificador en un conjunto de validación.

### Implementación de la función de extract_features.
* Esta función admite un solo tweet.
* Procesaremos el tweet usando la función `process_tweet()` importada y la guardaremos en la lista de palabras del tweet.
* Recorreremos cada palabra en la lista de palabras procesadas
     * Para cada palabra, consultaremos el diccionario `freqs` para el recuento cuando esa palabra tiene una etiqueta positiva '1'. (con clave (palabra, 1.0)
     * Hacemos lo mismo con el recuento para cuando la palabra esté asociada con la etiqueta negativa '0'. (con la clave (palabra, 0.0).)


In [88]:
freqs.get(('followfriday',1.0), 1)


23

In [86]:
def extract_features(tweet, freqs):
    # procesar el tweet (tokenizar, stems, remover stopwords, regex)
    word_l = process_tweet(tweet)

    #Crear el vector x (1x3)
    x = np.zeros((1,3))
    #bias en 1
    x[0,0]=1

    for word in word_l:
      #El conteo para los tokens que vienen de un tweet positivo
      x[0,1] += freqs.get((word,1.0), 1)

      #El conteo de los tokens que vienen de un tweet negativo
      x[0,2] += freqs.get((word,0.0),0)

    assert(x.shape==(1,3))
    return x

In [87]:
# checar la función
tmp1 = extract_features(train_x[0], freqs)
print(tmp1)

[[1.00e+00 3.02e+03 6.10e+01]]


# Implementación de la Regresión Logística


### Función sigmoide
Aprenderá a utilizar la regresión logística para la clasificación de texto.
* La función sigmoidea se define como:

$$ h(z) = \frac{1}{1+\exp^{-z}} \tag{1}$$

Asigna la entrada 'z' a un valor que varía entre 0 y 1, por lo que puede tratarse como una probabilidad.

<div style="width:image width px; font-size:100%; text-align:center;"><img src='https://github.com/gdesirena/Procesamiento_Natural_del_Lenguaje_2024/blob/main/Modulo%20II/Figures/sigmoid_plot.jpg?raw=1' alt="alternate text" width="width" height="height" style="width:300px;height:200px;" /> </div>

In [89]:
def sigmoid(z):
    h = 1 / (1 + np.exp(-z))
    return h

In [90]:
# probar la función
sigmoid(0)

0.5

In [91]:
sigmoid(4.58)

0.9898491991140074

In [92]:
sigmoid(-4.58)

0.01015080088599272

### Logistic regression: regression y función sigmoide

La regresión logística toma una regresión lineal regular y aplica un sigmoide a la salida de la regresión lineal.

Regresion:
$$z = \theta_0 x_0 + \theta_1 x_1 + \theta_2 x_2 + ... \theta_N x_N$$
Tenga en cuenta que los valores $ \theta $ son "pesos". Si realizó la especialización en aprendizaje profundo, nos referimos a los pesos con el vector `w`. En este curso, usamos una variable diferente $ \theta $ para referirnos a los pesos.

Regresión logística
$$ h(z) = \frac{1}{1+\exp^{-z}}$$
$$z = \theta_0 x_0 + \theta_1 x_1 + \theta_2 x_2 + ... \theta_N x_N$$
Nos referiremos a 'z' como los 'logits'.

### Función de costo y gradiente

La función de costo utilizada para la regresión logística es el promedio de la pérdida de registro en todos los ejemplos de entrenamiento:

$$J(\theta) = -\frac{1}{m} \sum_{i=1}^m y^{(i)}\log (h(z(\theta)^{(i)})) + (1-y^{(i)})\log (1-h(z(\theta)^{(i)}))\tag{5} $$
* $m$ es la cantidad de ejemplos de entrenamiento
* $y^{(i)}$ es la etiqueta real del i-ésimo dato de entrenamiento.
* $h(z(\theta)^{(i)})$ es la predicción del modelo para el i-ésimo ejemplo de entrenamiento.

La función de pérdida para un solo ejemplo de entrenamiento es
$$ Loss = -1 \times \left( y^{(i)}\log (h(z(\theta)^{(i)})) + (1-y^{(i)})\log (1-h(z(\theta)^{(i)})) \right)$$

* Todos los valores de $ h $ están entre 0 y 1, por lo que los registros serán negativos. Esa es la razón del factor -1 aplicado a la suma de los dos términos de pérdida.
* Tenga en cuenta que cuando el modelo predice 1 ($ h (z (\theta)) = 1 $) y la etiqueta $ y $ también es 1, la pérdida para ese ejemplo de entrenamiento es 0.
* De manera similar, cuando el modelo predice 0 ($ h (z (\theta)) = 0 $) y la etiqueta real también es 0, la pérdida para ese ejemplo de entrenamiento es 0.
* Sin embargo, cuando la predicción del modelo es cercana a 1 ($ h (z (\theta)) = 0.9999 $) y la etiqueta es 0, el segundo término de la pérdida logarítmica se convierte en un gran número negativo, que luego se multiplica por el factor general de -1 para convertirlo en un valor de pérdida positivo. $ -1 \times (1 - 0) \times log (1 - 0.9999) \approx 9.2 $ Cuanto más se acerque la predicción del modelo a 1, mayor será la pérdida.

* Del mismo modo, si el modelo predice cerca de 0 ($ h (z) = 0.0001 $) pero la etiqueta real es 1, el primer término en la función de pérdida se convierte en un número grande: $ -1 \times log (0.0001) \approx 9.2 $. Cuanto más cercana sea la predicción a cero, mayor será la pérdida.

In [93]:
#verificar que cuando el modelo predice cerca de 0 pero la etiqueta real es 1, la pérdida es un valor positivo grande
-1 * np.log(0.0001)

9.210340371976182

#### Actualizar los pesos

Para actualizar su vector de peso $ \theta $, aplicará el descenso de gradiente para mejorar iterativamente las predicciones de su modelo.
El gradiente de la función de costo $ J $ con respecto a uno de los pesos $ \theta_j $ es:

$$\nabla_{\theta_j}J(\theta) = \frac{1}{m} \sum_{i=1}^m(h^{(i)}-y^{(i)})x_j \tag{5}$$
* 'i' es el índice de todos los ejemplos de formación "m".
* 'j' es el índice del peso $ \theta_j $, entonces $ x_j $ es la característica asociada con el peso $ \theta_j $

* Para actualizar el peso $ \theta_j $, lo ajustamos restando una fracción del gradiente determinado por $ \alpha $:
$$ \theta_j = \theta_j - \alpha \times \nabla_{\theta_j} J (\theta) $$
* La tasa de aprendizaje $ \alpha $ es un valor que elegimos para controlar qué tan grande será una sola actualización.


## Implementación de la función Gradiente Descendente
* El número de iteraciones `num_iters` es el número de veces que utilizará todo el conjunto de entrenamiento.
* Para cada iteración, calculará la función de costo usando todos los ejemplos de entrenamiento (hay ejemplos de entrenamiento `m`), y para todas las funciones.
* En lugar de actualizar un solo peso $ \theta_i $ a la vez, podemos actualizar todos los pesos en el vector de columna:  
$$\mathbf{\theta} = \begin{pmatrix}
\theta_0
\\
\theta_1
\\
\theta_2
\\
\vdots
\\
\theta_n
\end{pmatrix}$$
* $ \mathbf {\theta} $ tiene dimensiones (n + 1, 1), donde 'n' es el número de características, y hay un elemento más para el término de sesgo $ \theta_0 $ (tenga en cuenta que el valor de característica correspondiente $ \mathbf {x_0} $ es 1).
* Los 'logits', 'z', se calculan multiplicando la matriz de características 'x' con el vector de peso 'theta'.  $z = \mathbf{x}\mathbf{\theta}$
    * $\mathbf{x}$ has dimensions (m, n+1)
    * $\mathbf{\theta}$: has dimensions (n+1, 1)
    * $\mathbf{z}$: has dimensions (m, 1)
* La predicción 'h' se calcula aplicando el sigmoide a cada elemento en 'z': $ h (z) = sigmoid (z) $, y tiene dimensiones (m, 1).
* La función de costo $ J $ se calcula tomando el producto escalar de los vectores 'y' y 'log (h)'. Dado que tanto 'y' como 'h' son vectores de columna (m, 1), transponga el vector a la izquierda, de modo que la multiplicación de matrices de un vector de fila con un vector de columna realice el producto escalar.
$$J = \frac{-1}{m} \times \left(\mathbf{y}^T \cdot log(\mathbf{h}) + \mathbf{(1-y)}^T \cdot log(\mathbf{1-h}) \right)$$
* La actualización de theta también está vectorizada. Debido a que las dimensiones de $ \mathbf {x} $ son (m, n + 1), y tanto $ \mathbf {h} $ como $ \mathbf {y} $ son (m, 1), necesitamos transponer $ \mathbf {x} $ y colóquelo a la izquierda para realizar la multiplicación de matrices, que luego da la respuesta (n + 1, 1) que necesitamos:
$$\mathbf{\theta} = \mathbf{\theta} - \frac{\alpha}{m} \times \left( \mathbf{x}^T \cdot \left( \mathbf{h-y} \right) \right)$$

In [94]:

def gradientDescent(x, y, theta, alpha, num_iters):
    m = x.shape[0]

    for i in range(0, num_iters):
      z = np.dot(x,theta)
      h = sigmoid(z)

      J = -1/m * (np.dot(y.transpose(), np.log(h)) + np.dot((1-y).transpose(), np.log(1-h)))

      theta = theta - (alpha/m)*np.dot(x.transpose(), (h-y))

      J = float(J)
    return J, theta

# Actividad 3: Modelo de Regresión Logística para Análisis de Sentimientos de tweets

### 1.- Entrenar el modelo

**instrucciones: Para entrenar el modelo:**

* Apile las características de todos los datos de entrenamiento en una matriz `X`.
* Llame la función `gradientDescent`, que se implementó anteriormente.



In [96]:
# recolectar todos los features apartir de la función extract_features
X = np.zeros((len(train_x),3))
for i in range(len(train_x)):
  X[i,:] = extract_features(train_x[i], freqs)

# Obtener las etiquetas reales (el valor de y) train_y
Y= train_y

# Aplicar el gradiente descendente
theta_0 = np.zeros((3,1)) # vector de 3x1 (inicializar aleatoriamente, o con zeros)
J, theta = gradientDescent(X,Y,theta_0, 1e-9, 1500)


In [97]:
theta

array([[ 7.25222017e-08],
       [ 5.23753412e-04],
       [-5.55251200e-04]])

### 2.- Probando la regresión logística

**Instrucciones para Predecir si un tweet es positivo o negativo**

* Dado un tweet, procéselo y luego extraiga las características.
* Aplicar los pesos aprendidos del modelo para obtener los logits.
* Aplicar la función sigmoide a los logits para obtener la predicción (un valor entre 0 y 1).

$$y_{pred} = sigmoid(\mathbf{x} \cdot \theta)$$

In [100]:
# UNQ_C4 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def predict_tweet(tweet, freqs, theta):

    #extraer los features con la función extract featurs X
    x = extract_features(tweet, freqs)

    #predecir utilizando la ecuacion de arriba ypred = sigmoid(np.dot(x, theta))
    y_pred = sigmoid(np.dot(x,theta))
    if y_pred >0.5:
      y_hat = 1
    else:
      y_hat = 0

    return y_pred, y_hat

In [101]:
tweets_test = ['I am happy', 'I am bad', 'This movie is amazing', 'great movie']
for tw in tweets_test:
  print(f'{tw} ---> {predict_tweet(tw, freqs,theta)}')

I am happy ---> (array([[0.51857391]]), 1)
I am bad ---> (array([[0.49433751]]), 0)
This movie is amazing ---> (array([[0.50356881]]), 1)
great movie ---> (array([[0.5153249]]), 1)


### 3.- Verificando el rendimiento del modelo

Después de entrenar su modelo con el conjunto de entrenamiento anterior, verifique cómo podría funcionar su modelo en datos reales no vistos probándolo con el conjunto de prueba.

**Instrucciones: Implementar `test_logistic_regression`**

* Dados los datos de prueba y los pesos de su modelo entrenado, calcule la precisión de su modelo de regresión logística.
* Utilice su función `predict_tweet ()` para hacer predicciones en cada tweet en el conjunto de prueba.
* Si la predicción es> 0,5, establezca la clasificación del modelo `y_hat` en 1; de lo contrario, establezca la clasificación del modelo` y_hat` en 0.
* Una predicción es precisa cuando `y_hat` es igual a` test_y`. Para calcular la precisón sume todas las instancias en las que ($y_hat==test_y$) sean iguales y divida por `m`.


In [ ]:
# UNQ_C5 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)None
def test_logistic_regression(test_x, test_y, freqs, theta):

    return accuracy

### 4.- Predice tu propio tweet

In [ ]:
# Feel free to change the tweet below
my_tweet = 'This is a ridiculously bright movie. The plot was terrible and I was sad until the ending!'
